# Geodatenanalyse 2


## Termin 10 - Modul 1

## Einführung in *Google Earth Engine* (*GEE*)

In [1]:
import ee
# initialize the connection to the server
ee.Initialize()
import geemap
import geemap.colormaps as cm
import datetime as dt
import pandas as pd

## Übung 1

In dieser Übung geht es um den Umgang mit Rastern (*ee.Image*). Hier werwenden wir das australische Höhenmodell von [Geoscience Australia](https://developers.google.com/earth-engine/datasets/catalog/AU_GA_DEM_1SEC_v10_DEM-S):


- **Aufgabe 1**: Welche Kanäle enthält dieses Raster? Welche weiteren Eigenschaften hat dieses Raster?

- **Aufgabe 2**: Zeige dieses Raster auf einer dynamischen Karte. 

- **Aufgabe 3**: Inspiziere die Karte manuell. Wo befindet sich der höchste Punkt auf dem australischen Kontinent? Wo der niedrigste?

In [2]:
# AUFGABE 1

au_dem = ee.Image("AU/GA/DEM_1SEC/v10/DEM-S")
bands = au_dem.bandNames().getInfo()
bands

['elevation']

In [3]:
# AUFGABE 1

au_dem.getInfo()

{'type': 'Image',
 'bands': [{'id': 'elevation',
   'data_type': {'type': 'PixelType', 'precision': 'float'},
   'dimensions': [147600, 122400],
   'crs': 'EPSG:4326',
   'crs_transform': [0.0002777777777822621,
    0,
    112.99986111110002,
    0,
    -0.0002777777777822621,
    -10.000138888696277]}],
 'id': 'AU/GA/DEM_1SEC/v10/DEM-S',
 'version': 1641990049596207,
 'properties': {'system:visualization_0_min': '-10.0',
  'type_name': 'Image',
  'keywords': ['australia',
   'dem',
   'elevation',
   'ga',
   'geophysical',
   'geoscience_australia',
   'smoothed',
   'srtm'],
  'thumb': 'https://mw1.google.com/ges/dd/images/DEMH_thumb.png',
  'description': '<p>The Smoothed Digital Elevation Model (DEM-S) was derived\nfrom the SRTM data acquired by NASA in February 2000. DEM-S represents\nground surface topography (excluding vegetation features) and has\nbeen smoothed to reduce noise and improve the representation of\nsurface shape. An adaptive process applied more smoothing in flatt

In [4]:
# AUFGABE 2

australia = geemap.Map()
vis_params = {'min': 0, 'max': 2300, 'palette': cm.palettes.dem}
australia.addLayer(au_dem, vis_params, 'SRTM DEM', True, 0.7)
australia.centerObject(au_dem)

australia

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

## Übung 2

Die globalen Waldtypen wurden mit Hilfe von Satellitenaufnahmen bestimmt und sind im [Copernicus Global Land Cover Layers](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_Landcover_100m_Proba-V-C3_Global) archiviert. Bearbeite folgende Aufgaben:

- **Aufgabe 1**: Welche Kanäle enthält diese Rasterkollektion? Welche Auflösung hat das Raster?

- **Aufgabe 2**: Welche Zeitpunkte sind für die Bestimmung vorhanden? Welche Waldtypen sind im Kanal 'forest_type' vorhanden?

- **Aufgabe 3**: Zeige das älteste Raster für den Waldtyp in einer interaktiven Karte. Beachte hierbei die Kategorien für Waldtypen und wähle eine Palette, mit welcher sich die Kategorien unterscheiden lassen. Welcher Waldtyp ist im Schwarzwald dominant?

In [5]:
# AUFGABE 1
land_cover = ee.ImageCollection("COPERNICUS/Landcover/100m/Proba-V-C3/Global")
land_cover.first().bandNames().getInfo()

['discrete_classification',
 'discrete_classification-proba',
 'bare-coverfraction',
 'urban-coverfraction',
 'crops-coverfraction',
 'grass-coverfraction',
 'moss-coverfraction',
 'water-permanent-coverfraction',
 'water-seasonal-coverfraction',
 'shrub-coverfraction',
 'snow-coverfraction',
 'tree-coverfraction',
 'forest_type',
 'data-density-indicator']

In [6]:
# AUFGABE 1

land_cover.first().select('forest_type').projection().nominalScale().getInfo()

100

In [7]:
# AUFGABE 2

dates = land_cover.aggregate_array("system:time_start")
datelist = pd.to_datetime(dates.getInfo(), unit='ms')
datelist

DatetimeIndex(['2015-01-01', '2016-01-01', '2017-01-01', '2018-01-01',
               '2019-01-01'],
              dtype='datetime64[ns]', freq=None)

In [8]:
# AUFGABE 2
forest = land_cover.first().select('forest_type')
forest.getInfo()

{'type': 'Image',
 'bands': [{'id': 'forest_type',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 255},
   'dimensions': [400750, 240450],
   'crs': 'EPSG:3857',
   'crs_transform': [100, 0, -20037550, 0, -100, 15538800]}],
 'id': 'COPERNICUS/Landcover/100m/Proba-V-C3/Global/2015',
 'version': 1612501859478524,
 'properties': {'forest_type_class_names': ['Unknown',
   'Evergreen needle leaf',
   'Evergreen broad leaf',
   'Deciduous needle leaf',
   'Deciduous broad leaf',
   'Mix of forest types'],
  'forest_type_class_palette': ['282828',
   '666000',
   '009900',
   '70663E',
   'A0DC00',
   '929900'],
  'copyright': 'Copernicus Service information 2020',
  'references': 'https://land.copernicus.eu/global/products/lc',
  'processing_level': 'L3',
  'source': 'Derived from EO satellite imagery',
  'platform': 'Proba-V',
  'discrete_classification_class_values': [0,
   20,
   30,
   40,
   50,
   60,
   70,
   80,
   90,
   100,
   111,
   112,


In [9]:
# AUFGABE 3

# Untersuche den Datensatz für die Kartierung ...
forest = ee.Image(land_cover.first().select('forest_type'))
forest.getInfo()['properties']['forest_type_class_names']

['Unknown',
 'Evergreen needle leaf',
 'Evergreen broad leaf',
 'Deciduous needle leaf',
 'Deciduous broad leaf',
 'Mix of forest types']

In [10]:
# AUFGABE 3

# erstelle eine interaktive Karte

globe = geemap.Map()

palette = forest.getInfo()['properties']['forest_type_class_palette']
vis_params = {'min': 0, 'max': 5, 'palette': palette}

globe.addLayer(forest, vis_params, 'SRTM DEM', True, 0.7)
globe.centerObject(forest)

colors = vis_params['palette']
vmin = vis_params['min']
vmax = vis_params['max']
globe.add_colorbar_branca(colors=colors, vmin=vmin, vmax=vmax, categorical=True, step=6)

globe

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [11]:
# AUFGABE 3

# bestimme den Waldtyp
forest.getInfo()['properties']['forest_type_class_names'][1]

'Evergreen needle leaf'

## ENDE